<a href="https://colab.research.google.com/github/itsarbababdurrab/Fake-and-Real-News-Detector/blob/main/Fake_and_Real_News_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# Load datasets
fake_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Random Projects/Real and Fake News Detector/Fake.csv')
true_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Random Projects/Real and Fake News Detector/True.csv')

In [12]:
# Add labels
fake_df['label'] = 0  # Fake News
true_df['label'] = 1  # Real News

In [13]:
# Combine datasets
df = pd.concat([fake_df, true_df], axis=0)
df = df.sample(frac=1).reset_index(drop=True)  # Shuffle dataset

In [14]:
df.head()

,title,text,subject,date,label
0,Native Americans move to frontlines in battle ...,"BELCOURT, North Dakota (Reuters) - Elvis Norqu...",politicsNews,"May 31, 2016",1
1,Costa Mesa Official: If You Don’t Make Enough...,California is one of the most expensive areas ...,News,"April 23, 2016",0
2,North Korea fires unidentified ballistic missi...,WASHINGTON (Reuters) - North Korea has fired a...,worldnews,"November 28, 2017",1
3,WATCH: Obama Directly Calls Trump Out For Car...,"While speaking at the G-7 Summit in Shima, Jap...",News,"May 26, 2016",0
4,U.S. diplomatic tiff with Russia should not be...,WASHINGTON (Reuters) - U.S. Secretary of State...,politicsNews,"September 12, 2017",1


In [15]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [20]:
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [21]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [22]:
def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    tokens = word_tokenize(text)  # Tokenization
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    tokens = [stemmer.stem(word) for word in tokens]  # Stemming
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatization
    return ' '.join(tokens)

In [23]:
df['processed_text'] = df['text'].apply(preprocess_text)

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
X_train, X_test, y_train, y_test = train_test_split(df['processed_text'], df['label'], test_size=0.2, random_state=42)

In [26]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pickle

In [28]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_tfidf, y_train)

RandomForestClassifier(random_state=42)

In [29]:
y_pred_rf = rf_model.predict(X_test_tfidf)

In [30]:
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 0.9976614699331848
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4657
           1       1.00      1.00      1.00      4323

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



In [31]:
pickle.dump(vectorizer, open("/content/vectorizer.pkl", "wb"))
pickle.dump(rf_model, open("/content/model.pkl", "wb"))

In [32]:
from google.colab import files

files.download("/content/vectorizer.pkl")
files.download("/content/model.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>